In [1]:
from ray.rllib.agents import ppo, a3c, dqn
from ray import tune
import numpy as np
import torch

import sys
sys.path.append("../..")
from thesis.utils.utils import get_config, setup_ray, save, load
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False)

In [2]:
env_args = dict(
    fleetsize = 2,
    max_fleetsize = 10,    
    pseudo_dispatcher = True,
    #pseudo_dispatcher_distance = 1,
    routing_agent_death= True,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 1, 
        reward_wrong_target = -0.5,
        reward_removed_for_block = -1, 
        #reward_target_distance = -0.05,
        reward_invalid= -0.2,
        block_timeout = 20,
        reward_accepted_in_station = 1,
        reward_declined_in_station = -1,
        dispatching_interval=360,
        io_quote = 0.99  ,
        availability = 0.95,
        mttr = 5*60,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = True,
        )
    )
)

In [4]:
config, logger_creator, checkpoint_dir = get_config(
    path = path,
    batch_size=1000,
    env_args = env_args, 
    agv_model = agv_model,
    train_agv = True,
    dispatcher_model=dispatcher_model, 
    train_dispatcher=True,
    env = "minimatrix",
    run_class="comparison",
    type = "dqn"
)
#trainer =  ppo.PPOTrainer(config, logger_creator=logger_creator)
#trainer = a3c.A3CTrainer(config, logger_creator=logger_creator)
trainer = dqn.DQNTrainer(config, logger_creator=logger_creator)

2022-11-12 00:49:46,789	INFO simple_q.py:187 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2022-11-12 00:49:46,791	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=460) c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_03478_1s0ejqem)
(RolloutWorker pid=460)   warn(f"Unzipping to temporary directory ({tmp_dir})")
(RolloutWorker pid=9928) c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_03478_g4b27w7p)
(RolloutWorker pid=9928)   warn(f"Unzipping to temporary directory ({tmp_dir})")
(RolloutWorker pid=460) 2022-11-12 00:50:14,658	WARNING

In [5]:
#trainer.restore("../../models/dqn_test/2_30_2022-11-09_22-11-07/checkpoint_001420/checkpoint-1420")

In [6]:
for j in range(10):
    for i in range(100):
        trainer.train()    
    trainer.save(checkpoint_dir)

2022-11-12 00:50:48,985	WARNING deprecation.py:46 -- DeprecationWarning: `ReplayBuffer.add_batch()` has been deprecated. Use `RepayBuffer.add()` instead. This will raise an error in the future!
2022-11-12 00:50:49,058	WARNING multi_agent_prioritized_replay_buffer.py:186 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
2022-11-12 00:50:49,067	WARNING deprecation.py:46 -- DeprecationWarning: `replay` has been deprecated. Use `sample` instead. This will raise an error in the future!


In [7]:
save(trainer, "agv", "../../models/trained")